# MPIA Arxiv on Deck 2: reprocessing

This notebook reprocesses some papers with the current baseline.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper to reprocess

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()

In [5]:
from glob import glob
paper_refs = [k.split('/')[-1].replace('.md',  '') for k in glob('_build/html/**md')]
n_papers = len(paper_refs)
print(f"Found {n_papers:,d} from past processing.")

Found 219 from past processing.


In [6]:
new_papers = [get_paper_from_identifier(k) for k in tqdm(paper_refs)]

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

  0%|          | 0/219 [00:00<?, ?it/s]

Arxiv has 219 new papers today
          191 with possible author matches


In [21]:
# move papers that are not matching

import os
import shutil

storage = "_build/html"

try:
    os.makedirs(os.path.join(storage, "removed"))
except FileExistsError:
    pass

for paper in new_papers:
    if paper not in candidates:
        print(f"moving {paper['identifier']} to {storage}/removed")
        shutil.move(os.path.join(storage, paper['identifier'] + '.md'),
                    os.path.join(storage, "removed", paper['identifier'] + '.md'))
        shutil.move(os.path.join(storage, "tmp_" + paper['identifier']),
                    os.path.join(storage, "removed", "tmp_" + paper['identifier']))

moving 2203.16735 to _build/html/removed
moving 2203.16856 to _build/html/removed
moving 2204.01758 to _build/html/removed
moving 2204.02109 to _build/html/removed
moving 2204.02998 to _build/html/removed
moving 2209.09823 to _build/html/removed
moving 2209.11096 to _build/html/removed
moving 2209.11781 to _build/html/removed
moving 2210.00863 to _build/html/removed
moving 2210.08078 to _build/html/removed
moving 2210.11103 to _build/html/removed
moving 2211.00667 to _build/html/removed
moving 2211.06454 to _build/html/removed
moving 2211.09217 to _build/html/removed
moving 2211.13318 to _build/html/removed
moving 2211.16996 to _build/html/removed
moving 2212.00047 to _build/html/removed
moving 2212.07332 to _build/html/removed
moving 2212.09307 to _build/html/removed
moving 2212.09443 to _build/html/removed
moving 2212.10804 to _build/html/removed
moving 2212.11991 to _build/html/removed
moving 2301.02265 to _build/html/removed
moving 2301.03387 to _build/html/removed
moving 2301.0736

# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [ ]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/190 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2203.15811
extracting tarball to tmp_2203.15811... done.
plugin pybtex.database.input.suffixes for suffix .bbl not found
Retrieving document from  https://arxiv.org/e-print/2203.15822
extracting tarball to tmp_2203.15822... done.


/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2203.15822/_region_.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: LatexWarning: attempting recovering figure figure/figure1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: LatexWarning: attempting recovering figure figure/figure3
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/tmp/ipykernel_12931/4289671964.py:41: LatexWarning: 2203.15822 did not run properly
Co

Retrieving document from  https://arxiv.org/e-print/2203.16504
extracting tarball to tmp_2203.16504... done.
plugin pybtex.database.input.suffixes for suffix .bbl not found
Retrieving document from  https://arxiv.org/e-print/2203.16734
extracting tarball to tmp_2203.16734... done.
list index out of range
Retrieving document from  https://arxiv.org/e-print/2203.16959
extracting tarball to tmp_2203.16959... done.


/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2203.16959/K2-2016-BLG-0005Lb.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:318: LatexWarning: Latex injecting: 'affils' from 'tmp_2203.16959/affils.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:318: LatexWarning: Latex injecting: 'authors' from 'tmp_2203.16959/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

✔ → 0:header
  ↳ 3939:\section{Introduction}
✔ → 3939:\section{Introduction}
  ↳ 10922:\section{Pristine Inner Galaxy Survey (PIGS)} \label{sect:pigs}
✘ → 10922:\section{Pristine Inner Galaxy Survey (PIGS)} \label{sect:pigs}
  ↳ 21322:\section{Fit and measure DIBs in stacked spectra} \label{sect:fit-dib}
✔ → 21322:\section{Fit and measure DIBs in stacked spectra} \label{sect:fit-dib}
  ↳ 31369:\section{Result} \label{sect:result}
✔ → 31369:\section{Result} \label{sect:result}
  ↳ 47425:\section{Discussion} \label{sect:discuss}
✔ → 47425:\section{Discussion} \label{sect:discuss}
  ↳ 54350:\section{Conclusions} \label{sect:conclusion}
✔ → 54350:\section{Conclusions} \label{sect:conclusion}
  ↳ 57753:\section{DIB fitting in stacked ISM spectra} \label{appsec:fit-stack}
✔ → 57753:\section{DIB fitting in stacked ISM spectra} \label{appsec:fit-stack}
  ↳ 58529:end
list index out of range
Retrieving document from  https://arxiv.org/e-print/2211.15288
extracting tarball to tmp_2211.15288... do

/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:392: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


✔ → 0:header
  ↳ 5712:\section{Introduction}
✔ → 5712:\section{Introduction}
  ↳ 13385:\section{Observations and Data Processing}\label{sec:sample}
✔ → 13385:\section{Observations and Data Processing}\label{sec:sample}
  ↳ 19291:\section{Absorption line catalog}\label{sec:method}
✔ → 19291:\section{Absorption line catalog}\label{sec:method}
  ↳ 59384:\section{Sample Properties}\label{sec:absorber_properties}
✔ → 59384:\section{Sample Properties}\label{sec:absorber_properties}
  ↳ 72593:\section{Completeness And False-Positive Rate}\label{sec:completeness}
✘ → 72593:\section{Completeness And False-Positive Rate}\label{sec:completeness}
  ↳ 94920:\section{Description of Released Products}\label{sec:catalog_desc}
✘ → 94920:\section{Description of Released Products}\label{sec:catalog_desc}
  ↳ 105580:\section{Summary}\label{sec:summary}
✔ → 105580:\section{Summary}\label{sec:summary}
  ↳ 112143:\section{Velocity shifting}\label{appendix:velshift}
✔ → 112143:\section{Velocity shifting}\labe

/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2212.00806/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.00811... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02816-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02816) | **Examining the Decline in the C IV Content of the Universe over 4.3 < z <  6.3 using the E-XQR-30 Sample**  |
|| <mark>Rebecca L. Davies</mark>, et al. -- incl., <mark>Sarah E. I. Bosman</mark>, <mark>Romain A. Meyer</mark>, <mark>Frederick B. Davies</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *20 pages, 8 figures, 4 tables. Published in MNRAS*|
|**Abstract**| Intervening CIV absorbers are key tracers of metal-enriched gas in galaxy halos over cosmic time. Previous studies suggest that the CIV cosmic mass density ($\Omega_{\rm CIV}$) decreases slowly over 1.5 $\lesssim z\lesssim$ 5 before declining rapidly at $z\gtrsim$ 5, but the cause of this downturn is poorly understood. We characterize the $\Omega_{\rm CIV}$ evolution over 4.3 $\lesssim z\lesssim$ 6.3 using 260 absorbers found in 42 XSHOOTER spectra of $z\sim$ 6 quasars, of which 30 come from the ESO Large Program XQR-30. The large sample enables us to robustly constrain the rate and timing of the downturn. We find that $\Omega_{\rm CIV}$ decreases by a factor of 4.8 $\pm$ 2.0 over the ~300 Myr interval between $z\sim$ 4.7 and $z\sim$ 5.8. The slope of the column density (log N) distribution function does not change, suggesting that CIV absorption is suppressed approximately uniformly across 13.2 $\leq$ log N/cm$^{-2}$ < 15.0. Assuming that the carbon content of galaxy halos evolves as the integral of the cosmic star formation rate density (with some delay due to stellar lifetimes and outflow travel times), we show that chemical evolution alone could plausibly explain the fast decline in $\Omega_{\rm CIV}$ over 4.3 $\lesssim z\lesssim$ 6.3. However, the CIV/CII ratio decreases at the highest redshifts, so the accelerated decline in $\Omega_{\rm CIV}$ at $z\gtrsim$ 5 may be more naturally explained by rapid changes in the gas ionization state driven by evolution of the UV background towards the end of hydrogen reionization. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02177-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02177) | **Data Combination: Interferometry and Single-dish Imaging in Radio  Astronomy**  |
|| Adele Plunkett, et al. -- incl., <mark>Lydia Moser-Fischer</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *29 pages, 20 figures. Accepted for publication in PASP. Code repository available at: github.com/teuben/DataComb*|
|**Abstract**| Modern interferometers routinely provide radio-astronomical images down to subarcsecond resolution. However, interferometers filter out spatial scales larger than those sampled by the shortest baselines, which affects the measurement of both spatial and spectral features. Complementary single-dish data are vital for recovering the true flux distribution of spatially resolved astronomical sources with such extended emission. In this work, we provide an overview of the prominent available methods to combine single-dish and interferometric observations. We test each of these methods in the framework of the CASA data analysis software package on both synthetic continuum and observed spectral data sets. We develop a set of new assessment tools that are generally applicable to all radio-astronomical cases of data combination. Applying these new assessment diagnostics, we evaluate the methods' performance and demonstrate the significant improvement of the combined results in comparison to purely interferometric reductions. We provide combination and assessment scripts as add-on material. Our results highlight the advantage of using data combination to ensure high-quality science images of spatially resolved objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02211-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02211) | **RR Lyrae Visual to Infrared Absolute Magnitude Calibrations. In the  light of Gaia DR3**  |
|| K. Looijmans, J. Lub, <mark>A.G.A. Brown</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *This paper reflects the presentation as given by J. Lub at the RRLCEP2022 conference (September 2022). Unfortunately after preparing this report we found out that only invited contributions would be published in the Proceedings*|
|**Abstract**| A probabilistic approach has been used in combination with the parallax data from Gaia (e)DR3 to calibrate Period-Luminosity-(Abundance) (PLZ) Relations covering a wide range of visual to Infrared observations of RR Lyrae stars. Absolute Magnitude Relations are given, derived from the same selection of stars, for $V$, $G$, $I$, $K_\mathrm{s}$ and WISE $W1$ as well as for for the reddening free pseudo-magnitudes $WBV$, $WVI$ and finally also Gaia $WG$. The classical relation between $M_V$ and [Fe/H] is redetermined and as an illustration distances are given to a few selected objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.03119-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.03119) | **Multiverse Predictions for Habitability: Stellar and Atmospheric  Habitability**  |
|| McCullen Sandora, Vladimir Airapetian, Luke Barnes, <mark>Geraint F. Lewis</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *23 pages, 2 figures*|
|**Abstract**| Stellar activity and planetary atmospheric properties have the potential to strongly influence habitability. To date, neither have been adequately studied in the multiverse context, so there has been no assessment of how these effects impact the probabilities of observing our fundamental constants. Here, we consider the effects of solar wind, mass loss, and extreme ultra-violet (XUV) flux on planetary atmospheres, how these effects scale with fundamental constants, and how this affects the likelihood of our observations. We determine the minimum atmospheric mass that can withstand erosion, maintain liquid surface water, and buffer diurnal temperature changes. We consider two plausible sources of Earth's atmosphere, as well as the notion that only initially slowly rotating stars are habitable, and find that all are equally compatible with the multiverse. We consider whether planetary magnetic fields are necessary for habitability, and find five boundaries in parameter space where magnetic fields are precluded. We find that if an Earth-like carbon-to-oxygen ratio is required for life, atmospheric effects do not have much of an impact on multiverse calculations. If significantly different carbon-to-oxygen ratios are compatible with life, magnetic fields must not be essential for life, and planet atmosphere must not scale with stellar nitrogen abundance, or else the multiverse would be ruled out to a high degree of confidence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02161-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02161) | **Exploring Fundamental Particle Acceleration and Loss Processes in  Heliophysics through an Orbiting X-ray Instrument in the Jovian System**  |
|| W. Dunn, et al. -- incl., <mark>W. Li</mark>, <mark>H. T. Smith</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *A White Paper for the 2024-2033 Solar and Space Physics (Heliophysics) Decadal Survey*|
|**Abstract**| Jupiter's magnetosphere is considered to be the most powerful particle accelerator in the Solar System, accelerating electrons from eV to 70 MeV and ions to GeV energies. How electromagnetic processes drive energy and particle flows, producing and removing energetic particles, is at the heart of Heliophysics. Particularly, the 2013 Decadal Strategy for Solar and Space Physics was to "Discover and characterize fundamental processes that occur both within the heliosphere and throughout the universe". The Jovian system offers an ideal natural laboratory to investigate all of the universal processes highlighted in the previous Decadal. The X-ray waveband has been widely used to remotely study plasma across astrophysical systems. The majority of astrophysical emissions can be grouped into 5 X-ray processes: fluorescence, thermal/coronal, scattering, charge exchange and particle acceleration. The Jovian system offers perhaps the only system that presents a rich catalog of all of these X-ray emission processes and can also be visited in-situ, affording the special possibility to directly link fundamental plasma processes with their resulting X-ray signatures. This offers invaluable ground-truths for astrophysical objects beyond the reach of in-situ exploration (e.g. brown dwarfs, magnetars or galaxy clusters that map the cosmos). Here, we show how coupling in-situ measurements with in-orbit X-ray observations of Jupiter's radiation belts, Galilean satellites, Io Torus, and atmosphere addresses fundamental heliophysics questions with wide-reaching impact across helio- and astrophysics. New developments like miniaturized X-ray optics and radiation-tolerant detectors, provide compact, lightweight, wide-field X-ray instruments perfectly suited to the Jupiter system, enabling this exciting new possibility. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02330-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02330) | **The Solar Cause of the 2022 February 3 Geomagnetic Storm that Led to the  Demise of the Starlink Satellites**  |
|| Nat Gopalswamy, <mark>Hong Xie</mark>, Seiji Yashiro, Sachiko Akiyama |
|*Appeared on*| *2023-03-07*|
|*Comments*| *14 pages, 8 figures, to appear in Sun and Geosphere*|
|**Abstract**| We report on the solar source of the 2022 February 3 geomagnetic storm of moderate strength that contributed to the loss of 39 Starlink satellites. The geomagnetic storm was caused by the 2022 January 29 halo coronal mass ejection (CME) that was of moderate speed (about 690 km/s) originating from NOAA active region 12936 located in the northeast quadrant (N18E06) of the Sun. The eruption was marked by an M1.1 flare, which started at 22:45 UT, peaked at 23:32 UT on January 29 and ended at 00:24 UT the next day. The CME ended up as a shock-driving magnetic cloud (MC) observed at Sun-Earth L1 and at STEREO-Ahead (STA) located about 34 deg behind Earth. The geomagnetic storm was caused by a strong southward component of the MC that was boosted by a high speed solar wind stream behind the MC. Even though Earth and STA were separated by only about 34 deg, the MC appeared quite different at Earth and L1. One possibility is that the MC was writhed reflecting the curved neutral line at the Sun. In-situ observations suggest that the MC was heading closer to STA than to Earth because of the earlier arrival at STA. However, the shock arrived at STA and Earth around the same time, suggesting a weaker shock at Earth due to flank passage. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02895) | **The Multiview Observatory for Solar Terrestrial Science (MOST)**  |
|| N. Gopalswamy, et al. -- incl., <mark>L. Li</mark>, <mark>J. Zhao</mark>, <mark>J. Zhang</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| *40 pages, 19 figures, 8 tables*|
|**Abstract**| We report on a study of the Multiview Observatory for Solar Terrestrial Science (MOST) mission that will provide comprehensive imagery and time series data needed to understand the magnetic connection between the solar interior and the solar atmosphere/inner heliosphere. MOST will build upon the successes of SOHO and STEREO missions with new views of the Sun and enhanced instrument capabilities. This article is based on a study conducted at NASA Goddard Space Flight Center that determined the required instrument refinement, spacecraft accommodation, launch configuration, and flight dynamics for mission success. MOST is envisioned as the next generation great observatory positioned to obtain three-dimensional information of solar wind structures such as coronal mass ejections, stream interaction regions, and the solar wind. The MOST mission consists of 2 pairs of spacecraft located in the vicinity of Sun Earth Lagrange points L4 (MOST1, MOST3) and L5 (MOST2 and MOST4). The spacecraft stationed at L4 (MOST1) and L5 (MOST2) will each carry seven remote-sensing and three in-situ instrument suites. MOST will also carry a novel radio package known as the Faraday Effect Tracker of Coronal and Heliospheric structures (FETCH). FETCH will have polarized radio transmitters and receivers on all four spacecraft to measure the magnetic content of solar wind structures propagating from the Sun to Earth using the Faraday rotation technique. The MOST mission will be able to sample the magnetized plasma throughout the Sun Earth connected space during the mission lifetime over a solar cycle. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.02952-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.02952) | **Pupil aberrations correction of the afocal telescope for the TianQin  project**  |
|| Zichao Fan, et al. -- incl., <mark>Lujia Zhao</mark> |
|*Appeared on*| *2023-03-07*|
|*Comments*| **|
|**Abstract**| TianQin is a planned Chinese space-based gravitational wave (GW) observatory with a frequency band of 10-4 to 1Hz. Optical telescopes are essential for the delivery of the measurement beam to support a precise distance measurement between pairs of proof masses. As the design is driven by the interferometric displacement sensitivity requirements, the stability control of optical path length (OPL) is extremely important beyond the traditional requirement of diffraction-limited imaging quality. In a telescope system, the recurring tilt-to-length (TTL) coupling noise arises from the OPL variation due to the wavefront deformation and angular misalignment. The pupil aberrations are preferred option to understand the OPL specifications and further suppress TTL coupling noise. To correct the pupil aberrations, we derive primary pupil aberrations in a series expansion form, and then refine the formulation of merit function by combining the pupil aberration theory and traditional image aberration theory. The automatic correction of pupil aberrations is carried out by using the macro programming in the commercial optical software Zemax, leading to a high performance telescope design. The design results show that on one side the pupil aberrations have been corrected, and on the other side, its optical performance meets the requirements for TianQin project. The RMS wavefront error over the science field of view (FOV) is less than {\lambda}/200 and the maximum TTL coupling noise over the entire 300 urad FOV is 0.0034nm/urad. We believe that our design approach can be a good guide for the space telescope design in any other space-based GW detection project, as well as other similar optical systems. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.02816.md
    + _build/html/tmp_2303.02816/./CIV_sightlines.png
    + _build/html/tmp_2303.02816/./completeness_CIV.png
    + _build/html/tmp_2303.02816/./C_ion_fractions_13.2_15.0.png
    + _build/html/tmp_2303.02816/./c_ion_modelling.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\OmCIV}{\mbox{\Omega_{\rm C   \textsc{iv}}}}$
$\newcommand{\OmCII}{\mbox{\Omega_{\rm C   \textsc{ii}}}}$
$\newcommand{\HII}{\mbox{H \textsc{ii}}}$
$\newcommand{\CI}{\mbox{C \textsc{i}}}$
$\newcommand{\CII}{\mbox{C \textsc{ii}}}$
$\newcommand{\CIII}{\mbox{C \textsc{iii}}}$
$\newcommand{\CIV}{\mbox{C \textsc{iv}}}$
$\newcommand{\CV}{\mbox{C \textsc{v}}}$
$\newcommand{\SiII}{\mbox{Si \textsc{ii}}}$
$\newcommand{\SiIV}{\mbox{Si \textsc{iv}}}$
$\newcommand{\NV}{\mbox{N \textsc{v}}}$
$\newcommand{\HeII}{\mbox{He \textsc{ii}}}$
$\newcommand{\FeII}{\mbox{Fe \textsc{ii}}}$
$\newcommand{\MgII}{\mbox{Mg \textsc{ii}}}$
$\newcommand{\OI}{\mbox{O \textsc{i}}}$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\dndx}{dn/dX}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\OmCIV}{\mbox{\Omega_{\rm C   \textsc{iv}}}}$
$\newcommand{\OmCII}{\mbox{\Omega_{\rm C   \textsc{ii}}}}$
$\newcommand{\HII}{\mbox{H \textsc{ii}}}$
$\newcommand{\CI}{\mbox{C \textsc{i}}}$
$\newcommand{\CII}{\mbox{C \textsc{ii}}}$
$\newcommand{\CIII}{\mbox{C \textsc{iii}}}$
$\newcommand{\CIV}{\mbox{C \textsc{iv}}}$
$\newcommand{\CV}{\mbox{C \textsc{v}}}$
$\newcommand{\SiII}{\mbox{Si \textsc{ii}}}$
$\newcommand{\SiIV}{\mbox{Si \textsc{iv}}}$
$\newcommand{\NV}{\mbox{N \textsc{v}}}$
$\newcommand{\HeII}{\mbox{He \textsc{ii}}}$
$\newcommand{\FeII}{\mbox{Fe \textsc{ii}}}$
$\newcommand{\MgII}{\mbox{Mg \textsc{ii}}}$
$\newcommand{\OI}{\mbox{O \textsc{i}}}$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\dndx}{dn/dX}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$</div>



<div id="title">

# Examining the Decline in the C IV Content of the Universe over \mbox{4.3 $\lesssim z \lesssim$ 6.3} using the E-XQR-30 Sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.02816-b31b1b.svg)](https://arxiv.org/abs/2303.02816)<mark>Appeared on: 2023-03-07</mark> - _20 pages, 8 figures, 4 tables. Published in MNRAS_

</div>
<div id="authors">

<mark><mark>Rebecca L. Davies</mark></mark>, et al. -- incl., <mark><mark>Sarah E. I. Bosman</mark></mark>, <mark><mark>Romain A. Meyer</mark></mark>, <mark><mark>Frederick B. Davies</mark></mark>

</div>
<div id="abstract">

**Abstract:** Intervening \CIV absorbers are key tracers of metal-enriched gas in galaxy halos over cosmic time. Previous studies suggest that the \CIV cosmic mass density ( \OmCIV ) decreases slowly over \mbox{1.5 $\lesssim z\lesssim$ 5} before declining rapidly at \mbox{$z\gtrsim$ 5} , but the cause of this downturn is poorly understood. We characterize the \OmCIV evolution over \mbox{4.3 $\lesssim z\lesssim$ 6.3} using 260 absorbers found in 42 XSHOOTER spectra of $z\sim$ 6 quasars, of which 30 come from the ESO Large Program XQR-30. The large sample enables us to robustly constrain the rate and timing of the downturn. We find that \OmCIV decreases by a factor of 4.8 $\pm$ 2.0 over the \mbox{$\sim$ 300 Myr} interval between $z\sim$ 4.7 and $z\sim$ 5.8. The slope of the column density ( $\log N$ ) distribution function does not change, suggesting that \CIV absorption is suppressed approximately uniformly across \mbox{13.2 $\leq\log N$/cm$^{-2}$ $<$ 15.0} . Assuming that the carbon content of galaxy halos evolves as the integral of the cosmic star formation rate density (with some delay due to stellar lifetimes and outflow travel times), we show that chemical evolution alone could plausibly explain the fast decline in \OmCIV over \mbox{4.3 $\lesssim z\lesssim$ 6.3} . However, the \CIV / \CII ratio decreases at the highest redshifts, so the accelerated decline in \OmCIV at $z\gtrsim$ 5 may be more naturally explained by rapid changes in the gas ionization state driven by evolution of the UV background towards the end of hydrogen reionization.

</div>

<div id="div_fig1">

<img src="tmp_2303.02816/./CIV_sightlines.png" alt="Fig4.1" width="50%"/><img src="tmp_2303.02816/./completeness_CIV.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Left: Illustration of the $\CI$V absorber sample. Horizontal lines show the redshift intervals over which the search for $\CI$V absorbers was conducted for each of the 42 quasar sightlines. Grey dashed regions highlight proximity zones within 10,000 $\kms$ of the quasar redshift. Longer gaps trace redshift intervals where the $\CI$V lines fall within BAL features or regions that were masked due to strong skyline or telluric contamination. The markers show all $\CI$V absorbers in the catalog, where solid circles indicate primary absorbers (those that were automatically identified, pass the visual inspection check, and do not fall in masked wavelength regions or BAL regions) and open squares indicate secondary absorbers (all others). Proximate absorbers are shown in black. The intervening absorbers are split into two redshift bins (indicated by the marker color), divided at the path-length-weighted mean redshift of our survey. Right: Completeness as a function of column density for $\CI$V absorbers in the two redshift intervals considered in \citetalias{Davies22Survey}. The dashed lines show the best-fit arctan functions published in that work. (*fig:completeness*)

</div>
<div id="div_fig2">

<img src="tmp_2303.02816/./C_ion_fractions_13.2_15.0.png" alt="Fig2" width="100%"/>

**Figure 2. -** Bar graph illustrating the contribution of $\CI$V and $\CI$I to the cosmic mass density of carbon in each redshift bin. Vertical lines illustrate the measurement errors and are centered on the path-length-weighted mean redshift of each bin with small offsets added for clarity. We are unable to measure $\OmCII$ in the two lowest redshift bins due to the saturation of the $\Lya$ forest. The evolving balance between $\OmCII$ and $\OmCIV$ across the two highest redshift bins suggests that changes in the UV background driven by hydrogen reionization may contribute to the decline in $\OmCIV$. (*fig:carbon_ion_fractions*)

</div>
<div id="div_fig3">

<img src="tmp_2303.02816/./c_ion_modelling.png" alt="Fig3" width="100%"/>

**Figure 3. -** Illustration of the possible ranges of gas densities probed by different ionization states of carbon at $z$ = 5 and $z$ = 6. The curves are the outputs of cloudy photoionization models. We fix the gas temperature to 10$^4$ K and adopt the 2011 update of the \citet{FaucherGiguere09} ionizing spectrum with a density-dependent correction for gas self-shielding as described in \citet{Keating16}. The ionizing spectrum is re-scaled at each redshift to match the H i photoionization rates measured by \citet{Calverley11}. (*fig:cloudy_modelling*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

5  publications in the last 7 days.
	 _build/html/2303.02816.md
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    

with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [17]:
print(page)

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <!-- Remove caching as much as possible -->
    <meta http-equiv="cache-control" content="no-cache" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />
    <!-- flickity bootstrap CSS -->
    <link rel="stylesheet" href="https://unpkg.com/flickity@2/dist/flickity.min.css">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
        integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <!-- highlight.js CSS -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
    <!-- flickity local CSS -->
    <link rel="stylesheet" href="flickity.css">
    <!-- markdown local CSS -->
    <link rel="sty